In [1]:
from anomalydetection.RDAE import RDAE
from anomalydetection.rpca import rpca
from anomalydetection.utils.ad import get_stats
from anomalydetection.utils.noise import add_noise
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import itertools
import tensorflow as tf
import heapq
from sklearn import preprocessing

In [2]:
def generate_gaussian(means, std_devs, dims, samples):
    assert len(means) == len(std_devs) == dims
    
    results = []
    for i in range(dims):
        t = np.random.normal(means[i], std_devs[i], samples)
        results.append(list(t))
    return np.array(list(zip(*results)))

In [3]:
def create_synth_data(dims, total_count, anomaly_perc, normal_means, normal_std_devs, anomalies_means, anomalies_std_devs):    
    normal_c = int(total_count * (1 - anomaly_perc))
    anomalies_c = total_count - normal_c 
    
    assert normal_c > 0
    assert anomalies_c > 0
    
    normal = generate_gaussian(normal_means, normal_std_devs, dims, normal_c)
    anomalies = generate_gaussian(anomalies_means, anomalies_std_devs, dims, anomalies_c)
    
    data = np.concatenate((normal, anomalies))
    labels = np.concatenate((np.repeat(0, normal_c), np.repeat(1, anomalies_c)))
    seed = np.random.randint(0, 100)
    rng = np.random.default_rng(seed)
    rng.shuffle(data, axis=0)
    rng = np.random.default_rng(seed)
    rng.shuffle(labels, axis=0)
    
    return data, labels

In [7]:
percentages = [1/5, 1/10, 1/15]
lambdas_rpca = np.linspace(0.000035, 0.01, 10)
lambdas_rdae = np.linspace(0.0001, 1, 10)
n_components = [1, 5, 10, 20]
dims = 20
total = 100



n_means = [3, 3.5, 4]
n_std_devs = [0.5, 0.7, 1]
a_means = [0]
a_std_devs = [0.5, 1]

tests = list(itertools.product(percentages, n_means, n_std_devs, a_means, a_std_devs))


s_rpca = []
s_pca = []
s_rdae = []
for anomaly_perc, nm, nsd, am, asd in tests:
    print(f'Contamination: {anomaly_perc}')
    print(f'Normal mean: {nm}')
    print(f'Normal stddev: {nsd}')
    print(f'Anomaly mean: {am}')
    print(f'Anomaly stddev: {asd}')

    normal_means = np.repeat(nm, dims)
    normal_std_devs =  np.repeat(nsd, dims)

    anomalies_means = np.repeat(am, dims)
    anomalies_std_devs = np.repeat(asd, dims)
    
    data, labels = create_synth_data(dims, total, anomaly_perc, normal_means, normal_std_devs, anomalies_means, anomalies_std_devs)
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(data)
    normalized_data = scaler.transform(data)
    
    """ 
    rpca_scores = []
    for l in lambdas_rpca:
        #print(f'lambda: {l}')
        
        colors = {0:'red', 1:'green'}
        
        model = rpca(verbose=False, lambda_=l, max_iter=200)
        L, S = model.fit_transform(data)
        
        stats = get_stats(S, labels, anomaly_perc)
        
        rpca_scores.append(round(stats['fscore'],4))
        
    print(rpca_scores)
    pca_scores = []
    for nc in n_components:
        #print(nc)
        pca = PCA(n_components=nc, svd_solver='auto')
        pca.fit_transform(data)

        x_redused = np.dot(data - pca.mean_, pca.components_.T)
        x_inversed = np.dot(x_redused, pca.components_) + pca.mean_
        
        stats = get_stats_(data - x_inversed, labels, anomaly_perc)
        
        pca_scores.append(round(stats['fscore'],4))
    
    print()
    """
    rdae_scores = []
    for l in lambdas_rdae:
        #print(l)
        model = RDAE(encoder_layer_size=[20, 10, 5], decoder_layer_size=[5, 10, 20], shrink='l21T',
            activation_function='sigmoid', regularizer='', verbose=False, lambda_=l)
        print(model.AE.summary())
        sleep(10)
        model.compile(loss='bce', optimizer=tf.keras.optimizers.Adam(0.0001))
        L, S = model.fit_transform(normalized_data, batch_size=10, inner_iteration=50, outer_iteration=10)
        
        stats = get_stats(S, labels, anomaly_perc)
        print(stats['fscore'])
        
        
        rdae_scores.append(round(stats['fscore'],4))
    
    #print(f'RPCA: {rpca_scores}')
    #print(f'PCA: {pca_scores}')
    print(f'RDAE: {rdae_scores}')
        
    #s_rpca.append(rpca_scores)
    #s_pca.append(pca_scores)
    s_rdae.append(rdae_scores)
    
    print('\n\n')
    
#s_rpca_np = np.array(s_rpca)
#s_pca_np = np.array(s_pca)
#s_rdae_np = np.array(s_rdae)
#with open('results/synthetic/rpca.npy', 'wb') as f:
#    np.save(f, s_rpca_np)
#with open('results/synthetic/pca.npy', 'wb') as f:
#    np.save(f, s_pca_np)
#with open('results/synthetic/rdae2.npy', 'wb') as f:
#    np.save(f, s_rdae_np)

Contamination: 0.2
Normal mean: 3
Normal stddev: 0.5
Anomaly mean: 0
Anomaly stddev: 0.5
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 10)                210       
                                                                 
 dense_15 (Dense)            (None, 5)                 55        
                                                                 
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________
None


ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [81]:
t = 4 #min(s_rpca_np.shape[1],s_pca_np.shape[1],s_rdae_np.shape[1])


for i in range(dim):
    
    print(s_rdae_np[i,np.argsort(-s_rdae_np[i])][:t])
    print(s_rpca_np[i,np.argsort(-s_rpca_np[i])][:t])
    print(s_pca_np[i][np.argsort(-s_pca_np[i])][:t])
    
    print('\n\n\n')

[0.35 0.3  0.3  0.3 ]
[1. 1. 1. 1.]
[1.   0.15 0.1  0.05]




[0.95 0.95 0.95 0.95]
[1. 1. 1. 1.]
[1.   1.   1.   0.85]




[0. 0. 0. 0.]
[1. 1. 1. 1.]
[1.   0.4  0.05 0.  ]




[0.75 0.7  0.7  0.65]
[1. 1. 1. 1.]
[1.   1.   0.85 0.55]




[0. 0. 0. 0.]
[1. 1. 1. 1.]
[0.6  0.15 0.   0.  ]




[0.4 0.4 0.3 0.3]
[1. 1. 1. 1.]
[0.95 0.25 0.15 0.05]




[0.25 0.25 0.2  0.2 ]
[1. 1. 1. 1.]
[1.   0.25 0.2  0.2 ]




[0.95 0.95 0.95 0.95]
[1. 1. 1. 1.]
[1.  1.  1.  0.8]




[0.05 0.   0.   0.  ]
[1. 1. 1. 1.]
[1.   0.5  0.05 0.  ]




[0.8  0.8  0.75 0.75]
[1. 1. 1. 1.]
[1.   1.   0.75 0.5 ]




[0. 0. 0. 0.]
[1. 1. 1. 1.]
[0.8  0.35 0.   0.  ]




[0.3  0.25 0.25 0.25]
[1. 1. 1. 1.]
[1.   0.15 0.15 0.1 ]




[0.3  0.25 0.25 0.25]
[1. 1. 1. 1.]
[1.   0.2  0.15 0.1 ]




[1. 1. 1. 1.]
[1. 1. 1. 1.]
[1.  1.  1.  0.8]




[0.05 0.05 0.   0.  ]
[1. 1. 1. 1.]
[1.   0.4  0.15 0.  ]




[0.85 0.8  0.75 0.75]
[1. 1. 1. 1.]
[1.   1.   0.9  0.35]




[0. 0. 0. 0.]
[1. 1. 1. 1.]
[0.9  0.25 0.   0.  ]




In [110]:
sum_pca = 0
sum_rdae = 0
sum_rpca = 0
for i in range(len(tests)):
    sum_rpca += s_rpca_np[i,np.argsort(-s_rpca_np[i])][0] == 1
    sum_rdae += s_rdae_np[i,np.argsort(-s_rdae_np[i])][0] == 1
    sum_pca += s_pca_np[i,np.argsort(-s_pca_np[i])][0] == 1


print("PCA:", end=' ')
print(sum_pca)
print("RDAE:", end=' ')
print(sum_rdae)
print("RPCA:", end=' ')
print(sum_rpca)

PCA: 45
RDAE: 5
RPCA: 54


In [121]:
indx = []
for i in range(len(tests)):
    if s_rdae_np[i,np.argsort(-s_rdae_np[i])][0] == 1:
        indx.append(i)
        
for i in indx:
    anomaly_perc, nm, nsd, am, asd = tests[i]
    
    normal_means = np.repeat(nm, dims)
    normal_std_devs =  np.repeat(nsd, dims)

    anomalies_means = np.repeat(am, dims)
    anomalies_std_devs = np.repeat(asd, dims)
    
    data, labels = create_synth_data(dims, total, anomaly_perc, normal_means, normal_std_devs, anomalies_means, anomalies_std_devs)
    
    #plt.scatter(data[:, 0], data[: ,1], label=labels, color=np.vectorize(colors.get)(labels))
    #plt.show()
    
    print( nm, nsd, am, asd)

4 0.5 0 1
3 0.5 0 1
3.5 0.5 0 1
4 0.5 0 1
3.5 0.5 0 1
